In [2]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Dict

# Initialize wandb
api = wandb.Api()

# Get runs from your project
runs = api.runs("go-embedding-evaluation")  # Replace with your actual project name



Detailed Results:
metric                                      Hits@1  Hits@10  Hits@5    MRR
base_ontology embedding_type model                                        
go-basic      anc2vec        MLPClassifier   0.035    0.322   0.185  0.123
              owl2vec        MLPClassifier   0.056    0.324   0.209  0.143
go-full       anc2vec        MLPClassifier   0.034    0.306   0.173  0.118
              biobert        MLPClassifier   0.016    0.155   0.077  0.066
              owl2vec        MLPClassifier   0.042    0.301   0.183  0.123


In [3]:

# Collect data
data = []
for run in runs:
    config = run.config
    # Get the last logged metrics
    history = run.history()
    if not history.empty:
        # Extract metrics for each embedding type and model
        for _, metrics in history.iterrows():
            metrics_dict = metrics.to_dict()
            for key, value in metrics_dict.items():
                if isinstance(value, (int, float)):  # Filter out non-numeric values
                    # Parse the metric key (format: "embedding_type/model/metric")
                    parts = key.split('/')
                    if len(parts) == 3:
                        embedding_type, model, metric = parts
                        if model == 'MLPClassifier':
                            data.append({
                                'base_ontology': config.get('base_ontology', 'unknown'),
                                'embedding_type': embedding_type,
                                'model': model,
                                'metric': metric,
                                'value': value
                            })
# Convert to DataFrame
df = pd.DataFrame(data)

# Create summary table
summary_table = df.pivot_table(
    index=['base_ontology', 'embedding_type', 'model'],
    columns='metric',
    values='value',
    aggfunc='last'
).round(3)

# Display the table
print("\nDetailed Results:")
print(summary_table)


Detailed Results:
metric                                      Hits@1  Hits@10  Hits@5    MRR
base_ontology embedding_type model                                        
go-basic      anc2vec        MLPClassifier   0.035    0.322   0.185  0.123
              owl2vec        MLPClassifier   0.056    0.324   0.209  0.143
go-full       anc2vec        MLPClassifier   0.034    0.306   0.173  0.118
              biobert        MLPClassifier   0.016    0.155   0.077  0.066
              owl2vec        MLPClassifier   0.042    0.301   0.183  0.123
